## Анализ базы данных сервиса для чтения книг по подписке<a class="tocSkip">

**Цель исследования:** проанализировать базу данных прибретенного компанией, крупного сервиса для чтения книг по подписке. Сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Исследуем таблицы, выведем первые строки

In [3]:
# таблица books
query = '''
            SELECT * 
            FROM books;
        '''

books  = pd.io.sql.read_sql(query, con = engine)

# таблица authors
query = '''
            SELECT * 
            FROM authors;
        '''

authors  = pd.io.sql.read_sql(query, con = engine)

# таблица publishers
query = '''
            SELECT * 
            FROM publishers;
        '''

publishers  = pd.io.sql.read_sql(query, con = engine)

# таблица ratings
query = '''
            SELECT * 
            FROM ratings;
        '''

ratings  = pd.io.sql.read_sql(query, con = engine)

# таблица reviews
query = '''
            SELECT * 
            FROM reviews;
        '''

reviews  = pd.io.sql.read_sql(query, con = engine)

In [4]:
display(books.head(), authors.head(), publishers.head(), ratings.head(), reviews.head())
display(books.info())
display(authors.info())
display(publishers.info())
display(ratings.info())
display(reviews.info())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

#### Описание данных

**Таблица `books`**
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
  
  
**Таблица `authors`**
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.
  
  
**Таблица `publishers`**
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
  
  
**Таблица `ratings`**
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
  
  
**Таблица `reviews`**
Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [5]:
# запрос
query = '''
            SELECT COUNT(DISTINCT book_id) 
            FROM books
            WHERE CAST(publication_date AS timestamp) > '2000-01-01'
        '''

books_amount = pd.io.sql.read_sql(query, con = engine)

In [6]:
books_amount

,count
0,819


**Вывод:** большая часть книг была выпущена после 1 января 2000 года - 819 из 1000 книг. 

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [7]:
# запрос
query = '''
            SELECT b.book_id,
                   COUNT(DISTINCT rw.review_id) AS reviews,
                   AVG(r.rating) AS avg_rating
            FROM books AS b
            LEFT OUTER JOIN ratings AS r ON b.book_id = r.book_id
            LEFT OUTER JOIN reviews AS rw ON b.book_id = rw.book_id  
            GROUP BY b.book_id;
        '''

rewiews_avg_rating = pd.io.sql.read_sql(query, con = engine)

In [8]:
rewiews_avg_rating

,book_id,reviews,avg_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


**Вывод:** все 1000 книг сервиса имеют обзоры и оценки пользователей, которые варьируются.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [9]:
# запрос
query = '''
            SELECT p.publisher,
                   COUNT(b.book_id) AS books_amount                   
            FROM (SELECT book_id,
                         publisher_id
                  FROM books
                  WHERE num_pages > 50) AS b
            LEFT OUTER JOIN publishers AS p ON b.publisher_id = p.publisher_id            
            GROUP BY p.publisher
            ORDER BY books_amount DESC
            LIMIT 1;
        '''

publisher_top_books_amount = pd.io.sql.read_sql(query, con = engine)

In [10]:
publisher_top_books_amount

,publisher,books_amount
0,Penguin Books,42


**Вывод:** Всего в сервисе 340 издательств, среди них издательство `Penguin Books` выспутило наибольшее количество книг - 42 книги, исключая брошюры. 

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [11]:
# запрос
query = '''
            SELECT a.author,
                   AVG(r.rating) AS avg_rating                  
            FROM (SELECT book_id,
                         rating 
                         FROM ratings
                         WHERE book_id IN (SELECT book_id
                                                  FROM ratings
                                                  GROUP BY book_id
                                                  HAVING COUNT(rating_id) > 50)
                  ) AS r 
            LEFT OUTER JOIN books AS b ON b.book_id = r.book_id
            LEFT OUTER JOIN authors AS a ON b.author_id = a.author_id
            GROUP BY a.author
            ORDER BY avg_rating DESC
            LIMIT 1;
        '''

author_top_rating = pd.io.sql.read_sql(query, con = engine)

In [12]:
author_top_rating

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** 636 авторов публикуют книги в сервисе, из них автор `J.K. Rowling/Mary GrandPré` наиболее популярна и имеет имеет самый высокий средний рейтинг 4.29 (среди книг с 50 и более оценками).

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
# запрос
query = '''
            SELECT AVG(r.review_amount) AS avg_review                  
            FROM (SELECT username,
                         COUNT(review_id) AS review_amount
                  FROM reviews
                  WHERE username IN (SELECT username
                                     FROM ratings 
                                     GROUP BY username
                                     HAVING COUNT(rating) > 50)
                  GROUP BY username) AS r;            
        '''

avg_reviews = pd.io.sql.read_sql(query, con = engine)

In [14]:
avg_reviews

,avg_review
0,24.333333


**Вывод:** всего пользователи в сервисе оставили 2793 обзора и 6456 оценок. Проанализировав активных пользователей (тех, которые поставили более 50 оценок) мы выяснили, что в среднем каждый такой пользователь оставляет 24 обзора на книги.

### Общий вывод

**Проведя краткий анализ, мы выяснили, что:**
- в сервисе публикуются довольно совеременные книги, большая часть из них была выпущена после 1 января 2000 года;
- все книги сервиса заинтересовали пользователей. Они охотно оставляют оценки книгам и пишут обзоры. Активные пользователи пишут в среднем по 24 обзора, что являтся хорошим показателем заинтересованности пользователей;
- самое активное издательство - `Penguin Books`(выпущено 42 книги);
- самый полярный автор - `J.K. Rowling/Mary GrandPré`(средний рейтинг 4.29).